In [1]:
import requests
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
#QUERY DATA

endpoint = f"https://public-api.defiyield.app/graphql/"
key = {"X-Api-Key" : "bad7014e-3fd4-473a-b981-1a5c2dd2d72e"}

def subQuery(pageNumber : int):
    query = """
    query {
        rekts (pageNumber: """+str(pageNumber)+""") {
            id
            projectName
            date
            fundsLost
            fundsReturned
            category
            issueType
        }
    }
    """
    result = requests.post(endpoint, json={"query": query}, headers=key)
    df=pd.DataFrame.from_dict(result.json()['data']['rekts'])
    return df


#construct full dataset and store
subDf = []
for i in range(1,64):
    subDf.append(subQuery(i))
df = pd.concat(subDf)
df['date']=pd.to_datetime(df.date, format='%Y-%m-%d')
df['fundsLost'] = pd.to_numeric(df.fundsLost)
df['fundsReturned'] = pd.to_numeric(df.fundsReturned)
df['count'] = 1
df.to_pickle('fullData_01')

In [ ]:
#LOAD LOCALLY

df = pd.read_pickle('fullData_01')
df.head()

In [ ]:
#FUNCTIONS FOR NICER VISUALISATIONS - ignore

#convert to postfix
def postfix(textStr, usd):
    usdStr = ''
    if usd:
        usdStr = '$'

    text_transform = (
            lambda x:
            f"{usdStr}{round(x//1000000000)}B"
                if x / 1000000000 >= 1
            else f"{usdStr}{round(x//1000000)}M"
                if x / 1000000 >= 1
            else f"{usdStr}{round(x//1000)}K"
                if x / 10000 >= 1
            else f"{usdStr}{round(x)}"
                if x >= 1
            else f"{usdStr}{round(x, 2)}"
        )
    
    return text_transform(float(textStr))

#annotate plot with postfix
def annot(plot, textStr, usd):
    if textStr == 'labels':
        for t in plot.texts:
            current_text = t.get_text()
            t.set_text(postfix(current_text, usd))

    if textStr == 'yticks':
        current_text = plot.get_yticks()
        new_text = [postfix(t, usd) for t in current_text]
        plot.set_yticklabels(new_text)

#order pivot table for heatmap presentation
def orderPivot(df):
    s = df.sum()
    ss = s.sort_values(ascending=False)
    df1 = df[ss.index]
    s2 = df.sum(axis=1)
    ss2 = s2.sort_values()
    df1 = df1.reindex(ss2.index)
    return df1
        

### First we recreate the cumulative sum of lost and recovered funds from the Rekt homepage.

In [ ]:
#Sum of lost and recovered funds
sumDf = df.copy()
sumDf.date = sumDf.date.dt.strftime('%Y-%m')

sumTable = pd.pivot_table(sumDf, values=['fundsLost','fundsReturned'], index='date', aggfunc=np.sum)
sumTable['lostSum'] = sumTable.fundsLost.cumsum()
sumTable['recoveredSum'] = sumTable.fundsReturned.cumsum()

sumPlot = sumTable.plot(y=['recoveredSum','lostSum'])
sumPlot.figure.set_size_inches(12,4)
annot(sumPlot,'yticks', usd=True)
sumPlot.set_title('Cumulative sum of lost and recovered funds')

### A nice ressemblance to the original. <br>
### Now we break these losses into the issues that caused them.

In [ ]:
#TIME SERIES

def timeSeries(df, value, categories, exp=False, usd=False, freq='Y'):
    
    timeFormat = '%Y'
    if freq == 'M':
        timeFormat = '%b '+timeFormat

    df['formatedDate'] = df.date.dt.strftime(timeFormat)

    table = pd.pivot_table(df, values=value, index='formatedDate', columns=categories, aggfunc=np.sum)

    dateRange = pd.period_range(start=min(df.date), end=max(df.date), freq=freq)
    table = table.reindex(dateRange.strftime(timeFormat), fill_value=0)

    plot = table.plot(kind='bar')
    
    if exp:
        plot.set_yscale('log')
    
    plot.figure.set_size_inches(12,4)
    annot(plot, 'yticks', usd)
    
    return plot

In [ ]:
#Break down into loss by issue
issuePlot = timeSeries(df=df.copy(), value='fundsLost', categories='issueType', exp=True, usd=True)
issuePlot.set_title('Funds lost over time by issue, exponential scale')

### Using an exponential scale to make the data above more comprehensible visually. <br>
### We note that before 2020 there were only rugpull, access control and unlabelled types of issues. Since then many different types have appeared, but none surpase the former in terms of total funds lost. <br>
### Finally to explain the peak of lost funds in 2022 to unlabelled issues 'Other', we consider the 10 biggest losses below and see a major contribution from the Terra Classic project of 40B USD.

In [ ]:
topLoss = df.copy()
topLoss['lostFunds'] = topLoss.fundsLost.apply(lambda x: postfix(x, usd=True))
topLoss = topLoss.sort_values(by='fundsLost',ascending=False).head(10)
topLoss[['projectName', 'date', 'lostFunds','category','issueType']]

In [ ]:
#Break down into loss by issue
issueDf = df[df['date']>datetime.strptime('2020-01-01','%Y-%m-%d')]
issueCount = timeSeries(df=issueDf, value='count', categories='issueType', exp=False, usd=False, freq='M')
issueCount.set_title('Number of breaches over time by issue')

### Above we see the number of attacks instead of the amount of funds lost. We shorten the range to Aug 2020 (there were very little before) and break-down to monthly data. We note a surprising jump in rugpull and honeypot attacks in late 2021 but no other obvious patterns of seasonality.

### Now we consider some of the possible relations between the different breaches. We look at heatmaps between the category of the project breached and the issue that caused it.

In [ ]:
#HEATMAP OF AGGREGATED VALUES BY TWO CATEGORICAL VARIABLES

def Heatmap(df, columns, index, value, usd=False, agg=np.sum):
    table = pd.pivot_table(df, values=value, index=index, columns=columns, aggfunc=agg)
    table = orderPivot(table)

    heatmap = sns.heatmap(table, cmap='Reds', annot=True, cbar=False, yticklabels=True)
    heatmap.figure.set_size_inches(8,12)
    
    annot(heatmap, 'labels', usd)

    return heatmap


In [ ]:
#HEATMAP OF TOTAL FUNDS LOST BY CATEGORY AND ISSUE TYPE
heatmap = Heatmap(df=df, columns='issueType', index='category', value='fundsLost', usd=True)
heatmap.set_title('Sum of lost funds by category and issue')

### Above we see that by far the largest combination is stablecoins with an unlabelled type of attack. Since such a large amount of funds are lost to these unlabelled 'Other' issues, it would be interesting to try to categorise them. An interesting thing to look at (for which we would need another source of data), would be the funds lost for each of these project categories as a proportion of the estimated funds currently in such projects, this could give an indication of the level of risk.

In [ ]:
#consider number of breaches instead of sum of lost funds
heatmap = Heatmap(df=df.copy(), columns='issueType', index='category', value='count', usd=False)
heatmap.set_title('Number of breaches by category and issue')

### Looking now at the number of breaches we find a perhaps surprising result, that token projects with the honeypot issue greatly outnumber all other combinations. Especially surprising since these only lost about 7M USD in total, far less than many others. <br>
### To look further into this we consider the average funds lost by each attack in the next heatmap, and we see that token/rugpull attacks lose on average a meager 4200 USD, compared to the billions lost in stablecoins or borrowing and lending.

In [ ]:
# average funds lost by attack
heatmap = Heatmap(df=df.copy(), columns='issueType', index='category', value='fundsLost', usd=True, agg=np.average)
heatmap.set_title('Average funds lost by category and issue')

### Finally we consider a breakdown of the funds that were recovered. Again this shows that further categorisation could be useful since the large majority of returned funds are unlabelled.

In [ ]:
#RECOVERED FUNDS BY CATEGORY AND ISSUE TYPE
df3 = df.loc[(df.fundsReturned!=0)]
heatmap = Heatmap(df=df3, columns='issueType', index='category', value='fundsReturned', usd=True)
heatmap.figure.set_size_inches(5,5)
heatmap.set_title('Sum of recovered funds by category and issue')

### By considering the average proportion of the funds that are recovered we see that most often, when funds are recovered, a large majority of the total funds are indeed recovered.

In [ ]:
#RECOVERED FUNDS BY CATEGORY AND ISSUE TYPE
fundsPropDf = df.loc[(df.fundsReturned!=0)]
fundsPropDf['returnedProp'] = pd.to_numeric(fundsPropDf.fundsReturned,downcast='float')/pd.to_numeric(fundsPropDf.fundsLost,downcast='float')
heatmap = Heatmap(df=fundsPropDf, columns='issueType', index='category', value='returnedProp', usd=True, agg=np.average)
heatmap.figure.set_size_inches(5,5)
heatmap.set_title('Average proportion of funds recovered (per $1) by category and issue')

### Takeaway: <br>
#### 1. There was a clear change in 2020 when many new breach types started occuring, and 2023 already looks set to match previous years. <br>
#### 2. A deeper classification of issues and categories could be interesting, for example the Terra Classic 40B USD breach could be classified as 'failed migration' or 'panic event'. Allowing classification of a breach to multiple issues could also be of value. <br>
#### 3. Breaking down the lost and recovered funds by categories shows some interesting patterns, and could definitely be helped by comparing these to the current state of live projects, if such data is available.